# Expressed genes within the Azhurnaya developmental time course (209 samples) and Chinese Spring no stress (123 samples).

This notebook is used to produce the table with the mean tpms grouped by the intermediate factors. 

The code corresponds to the methods stated as: 

"Starting from the subset of genes considered expressed using the initial 850 filter criterion, we determined genes which were expressed in at least one tissue within the Azhurnaya developmental time course (209 samples; 22 intermediate tissues) and Chinese Spring no stress (123 samples; 15 intermediate tissues) datasets. For this analysis, we first calculated the average TPM expression of each gene in each of the intermediate tissue types (average expression per tissue). The number of samples that went into generating this average expression per tissue value varied for each intermediate tissue and are available in Table S1.

"We considered a gene expressed when its average expression per tissue was > 0.5 TPM in at least one intermediate tissue. For both datasets we focused on HC gene models (10). Whilst expression data was also assessed for LC genes, we excluded these from the main analysis to avoid confounding effects from pseudogenes and low-quality gene models. Through this analysis we found evidence of expression for 83,741 (75.6%) HC genes in Azhurnaya and 82,567 (74.5%) HC genes in Chinese Spring.

Using the average expression per tissue values, we also determined the global expression of each gene across all tissues in which it was expressed (based on the > 0.5 TPM criteria in the tissue). This generated an average value across tissues, rather than a geometric mean across all samples, to account for the variation in the number of samples per tissue. It also excludes tissues in which a gene is not expressed. This average across expressed tissues is referred to as either the “global analysis” or the “combined analysis (all tissues)” across the main text and in the supplementary materials and tables."




In [1]:
library(sqldf)
library(ggplot2)
library(reshape2)
library(fields)
library("gridExtra")
library(ggtern)
library(clue)
library(geometry)
require(gtable)

Loading required package: gsubfn
Loading required package: proto
Loading required package: RSQLite
Loading required package: spam
Loading required package: grid
Spam version 1.4-0 (2016-08-29) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.

Attaching package: ‘spam’

The following objects are masked from ‘package:base’:

    backsolve, forwardsolve

Loading required package: maps
--
Consider donating at: http://ggtern.com
Even small amounts (say $10-50) are very much appreciated!
Remember to cite, run citation(package = 'ggtern') for further info.
--

Attaching package: ‘ggtern’

The following objects are masked from ‘package:gridExtra’:

    arrangeGrob, grid.arrange

The following objects are masked from ‘package:ggplot2’:

    %+%, aes, annotate, calc_element, ggplot, ggplot_build,
    ggplot_gtable, ggplot

In [2]:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                metadata<-read.csv("/Users/ramirezr/Dropbox/JIC/expVIPMetadatas/RefSeq1.0/metadatas/Metadata_june11.txt", row.names = 1, sep="\t")
metadata<-read.csv("./TablesForExploration/metadata.csv")

nrow(metadata)
loadValuesFromExperiment<-function(metadata, folder, unit="tpm", values=c("Development")){
    metadata$Sample.IDs <- gsub("-",".",metadata$Sample.IDs)
    
    v<-values[1]
    v<-gsub(" ","_",v)
    v<-gsub(",",".",v)
    path<-paste0(folder,"/",v,"_",unit,".tsv")
    ret<-read.table(path, row.names = 1, header= TRUE)
    if(length(values) > 1){
      for(i in 2:length(values)){
        v<-values[i]
        v<-gsub(" ","_",v)
        v<-gsub(",",".",v)
        path<-paste0(folder,"/",v,"_",unit,".tsv")
        tmp<-read.table(path, row.names = 1, header= TRUE)
        ret<-cbind(ret,tmp)
      }  
    }
    
    md<-metadata[metadata$Sample.IDs%in%colnames(ret),]
    ret<-ret[,as.character(md$Sample.IDs),]
    list(ret,md)
}
folder<-"./expressionValuesPerGene"
tpms  <-loadValuesFromExperiment(metadata, folder, unit="tpm",  values=unique(metadata$study_title))


metadata_used<-tpms[[2]]
tpms<-tpms[[1]]

nrow(metadata_used)
head(metadata_used)




[1] 850

[1] 850

Sample.IDs,scientific_name,study_title,High.level.variety,High.level.tissue,High.level.age,High.level.stress.disease,Variety,Tissue,Age,Stress.disease,Comments,DOI,Intermediate,Intermediate_Stress_merged_control,Intermediate_Stress
Sample_10,Triticum aestivum,Development,BCS cv-1,roots,seedling,none,BCS cv-1,radicle,Seedling stage,none,,,roots,NA,NA
Sample_18,Triticum aestivum,Development,BCS cv-1,roots,seedling,none,BCS cv-1,radicle,Seedling stage,none,,,roots,NA,NA
Sample_3A,Triticum aestivum,Development,BCS cv-1,roots,seedling,none,BCS cv-1,radicle,Seedling stage,none,,,roots,NA,NA
Sample_12,Triticum aestivum,Development,BCS cv-1,leaves/shoots,seedling,none,BCS cv-1,coleoptile,Seedling stage,none,,,seedling aerial tissues,NA,NA
Sample_26,Triticum aestivum,Development,BCS cv-1,leaves/shoots,seedling,none,BCS cv-1,coleoptile,Seedling stage,none,,,seedling aerial tissues,NA,NA
Sample_6A,Triticum aestivum,Development,BCS cv-1,leaves/shoots,seedling,none,BCS cv-1,coleoptile,Seedling stage,none,,,seedling aerial tissues,NA,NA


In [3]:
expressed_genes<-read.csv("./TablesForExploration/expressed_genes_tpmsOver0.5AtLeast8Samples.csv")
colnames(expressed_genes) <- c("gene","tpm", "count")
genes_to_use <- data.frame(gene=expressed_genes$gene)
head(genes_to_use)
nrow(genes_to_use)

gene
TraesCS1A01G000100
TraesCS1A01G000100LC
TraesCS1A01G000200
TraesCS1A01G000200LC
TraesCS1A01G000300
TraesCS1A01G000300LC


[1] 172034

In [4]:
getSamplesForFactor<- function(metadata, type="High.level.tissue",factor="roots"){
    ret<-""
    if(type != "all"){
        ret<-as.character(metadata[metadata[,type] == factor,]$Sample.IDs)
    }else{
        ret<-unique(metadata$Sample.IDs)
    }
    ret
}
getMeansPerFactor<- function(values, metadata,  type="High.level.tissue",factor="roots"){
    samples <- getSamplesForFactor(metadata, type, factor)
    vals <- values[,samples]
    mean<-0
    if(length(samples) == 1){
        print("This factor only has one sample!")
        print(factor)
        mean<-vals
    }else{
        mean<-rowMeans(vals)
    }
    
    
    mean<-sort(mean,decreasing=T)
    cumulative <- cumsum(mean)
    
    cumulative<-data.frame(cumulative)
    mean<-data.frame(mean)
    
    mean$gene <- rownames(mean)
    cumulative$gene <- rownames(cumulative)
    mean$total_samples <- length(samples)
    
    n <-merge(mean,cumulative, by='gene', all=T)
    n <- n[order(n$cumulative,decreasing = F),]
    n$seq <- seq(from = 1, to = nrow(n))
    n$factor = factor
    n
}
getMeansForAllFactors<-function(values, metadata,  type="High.level.tissue"){
    factors<-unique(metadata[,type])
    f<-factors[1]
    meansDFs <- getMeansPerFactor(tpms,metadata,type=type, factor=f)
    for (i in 2:length(factors)){
        f<-factors[i]
        localDF<-getMeansPerFactor(tpms,metadata,type=type, factor=f)
        meansDFs <- rbind(meansDFs,localDF)
    }
    meansDFs
}

isExpressedPerFactor <- function(values, metadata,  type="High.level.tissue",factor="roots", minTPM=0.5){
    samples <- getSamplesForFactor(metadata, type, factor)
    vals <- values[,samples]
    means <- rowMeans(vals)
    expr <- means > minTPM
    m2 <- data.frame( expressed = expr)
    m2$factor<-factor
    m2$transcript<-rownames(m2)
    m2$total_samples <- length(samples)
    m2
}

getExclusiveExpression<-function(values, metadata, minTPM=0.5, type="High.level.tissue"){
    means <- getMeansForAllFactors(values, metadata,type=type)
    means$expressed<-means$mean > minTPM
    exclusiveExpresison<-sqldf("SELECT gene, factor, mean, total_samples 
        FROM means 
        WHERE expressed 
        GROUP BY gene HAVING count(factor) = 1 ")
   list(means, exclusiveExpresison )
}



In [5]:
get_means_df<-function(metadata, tpms, type="High.level.tissue", min_mean_tpm=0.5){
    samples<-getSamplesForFactor(metadata, type="all",factor="all")
    values<-data.frame(value=numeric(nrow(tpms)),stringsAsFactors=FALSE)
    if(length(samples) > 1){
        values$value<-rowMeans(tpms[,samples])
    }else{
        values$value<-tpms[,samples]
    }
     
    values$factor<-"all"
    values$gene<-rownames(tpms)
    values$samples<-length(samples)
    
    #print(unique(metadata[,type]))
    
    for(f in unique(metadata[,type])){
        #print(f)
        samples<-getSamplesForFactor(metadata, type=type,factor=f)
        
        tmp<-data.frame(value=numeric(nrow(tpms)),stringsAsFactors=FALSE)
        
        if(length(samples) > 1){
            tmp$value<-rowMeans(tpms[,samples])
        }else{
            tmp$value<-tpms[,samples]
        }
        
        tmp$factor<-f
        tmp$gene<-rownames(tpms)
        tmp$samples<-length(samples)
        values<-rbind(values,tmp)
    }
    
    casted<-dcast(values, gene~factor, value.var="value")
    casted$all<-NULL
   
    rownames(casted)<-casted$gene
    casted$gene<-NULL
    casted<-as.matrix(casted)
 
    tmp<-data.frame(value=numeric(nrow(casted)),stringsAsFactors=FALSE)
    tmp$value<-rowMeans(casted)
    tmp$factor<-"all_means"
    tmp$gene<-rownames(casted)
    tmp$samples<-ncol(casted)
    values<-rbind(values,tmp)
    
    casted<-ifelse(casted < min_mean_tpm, NA, casted)
    
    tmp<-data.frame(value=numeric(nrow(casted)),stringsAsFactors=FALSE)
    tmp$value<-rowMeans(casted, na.rm = TRUE)
   
    tmp$factor<-"all_mean_filter"
    tmp$gene<-rownames(casted)
    tmp$samples<-rowSums(!is.na(casted))
    
    values<-rbind(values,tmp)
    values
}

means_df<-get_means_df(metadata_used, tpms)
head(means_df)

value,factor,gene,samples
27.93595952,all,TraesCS1A01G000100,850
965.05451894,all,TraesCS1A01G000100LC,850
196.33845519,all,TraesCS1A01G000200,850
0.06421839,all,TraesCS1A01G000200LC,850
28.23783663,all,TraesCS1A01G000300,850
1.06279369,all,TraesCS1A01G000300LC,850


In [6]:
unique(means_df$factor)

[1] "all"             "roots"           "leaves/shoots"   "spike"          
[5] "grain"           "all_means"       "all_mean_filter"

In [7]:
head(means_df[means_df$factor=='all_mean_filter',])

,value,factor,gene,samples
1617499,23.102005,all_mean_filter,TraesCS1A01G000100,4
1617500,957.867961,all_mean_filter,TraesCS1A01G000100LC,4
1617501,175.665915,all_mean_filter,TraesCS1A01G000200,4
1617502,NaN,all_mean_filter,TraesCS1A01G000200LC,0
1617503,21.807164,all_mean_filter,TraesCS1A01G000300,4
1617504,3.088701,all_mean_filter,TraesCS1A01G000300LC,2


In [8]:
group_to_execute<-read.csv("./TablesForExploration/samples_to_use.csv")
head(group_to_execute)
tail(group_to_execute)

Sample.IDs,subset,type_to_use
Sample_10,Development,Intermediate
Sample_18,Development,Intermediate
Sample_3A,Development,Intermediate
Sample_12,Development,Intermediate
Sample_26,Development,Intermediate
Sample_6A,Development,Intermediate


,Sample.IDs,subset,type_to_use
2718,ERR789107,stress,Intermediate_Stress
2719,ERR789106,stress,Intermediate_Stress
2720,ERR789105,stress,Intermediate_Stress
2721,ERR789108,stress,Intermediate_Stress
2722,ERR789110,stress,Intermediate_Stress
2723,ERR789109,stress,Intermediate_Stress


The loop callos the function ```get_means_df``` accross different subsets of data and according to the Intermediate grouping factors. There are three special global averages:

 1. ```all``` Average across all the samples
 2. ```all_mean``` Average of averages per factor
 3. ```all_mean_filter``` Average of average per factor, but only when the average of each grouping is  ```> 0.5``` (as per ```min_mean_tpm```)

In [9]:
means_df <- NULL
for(subset in unique(group_to_execute$subset)){
    samples_in_group<-group_to_execute[group_to_execute$subset == subset,]
    column <- unique(samples_in_group$type_to_use)
    if(length(column) != 1){
        print(paste0(subset, " has more than one possible column"))
    }
    print(subset)
    print(as.character(column))
    local_metadata <- metadata[metadata$Sample.IDs %in% samples_in_group$Sample.IDs,]
    print(nrow(local_metadata))
    tmp_df <- get_means_df(local_metadata, tpms, type=as.character(column), min_mean_tpm=0.5)
    tmp_df$subset<-subset
    tmp_df$min_mean_tpm<-as.factor("0.5")
    
    if(is.null(means_df)){
        means_df <- tmp_df
    }else{
        means_df<-rbind(tmp_df, means_df)
    }
}


[1] "Development"
[1] "Intermediate"
[1] 209
[1] "850_samples"
[1] "Intermediate"
[1] 850
[1] "CS_no_stress"
[1] "Intermediate"
[1] 123
[1] "CS_NB_inc_stress"
[1] "Intermediate"
[1] 144
[1] "abiotic"
[1] "Intermediate_Stress"
[1] 50
[1] "disease"
[1] "Intermediate_Stress"
[1] 163
[1] "grain"
[1] "Intermediate"
[1] 119
[1] "leaf"
[1] "Intermediate"
[1] 245
[1] "root"
[1] "Intermediate"
[1] 45
[1] "spike"
[1] "Intermediate"
[1] 128
[1] "abiotic_merged_control"
[1] "Intermediate_Stress_merged_control"
[1] 50
[1] "disease_merged_control"
[1] "Intermediate_Stress_merged_control"
[1] 163
[1] "stress_control"
[1] "Intermediate_Stress"
[1] 77
[1] "abiotic_stress_control"
[1] "Intermediate_Stress"
[1] 13
[1] "disease_stress_control"
[1] "Intermediate_Stress"
[1] 64
[1] "abiotic_stress"
[1] "Intermediate_Stress"
[1] 34
[1] "disease_stress"
[1] "Intermediate_Stress"
[1] 106
[1] "stress"
[1] "Intermediate_Stress"
[1] 140


In [10]:
nrow(means_df)
head(means_df)

[1] 69552414

value,factor,gene,samples,subset,min_mean_tpm
40.6321681,all,TraesCS1A01G000100,140,stress,0.5
1247.8227773,all,TraesCS1A01G000100LC,140,stress,0.5
357.8591670,all,TraesCS1A01G000200,140,stress,0.5
0.1167718,all,TraesCS1A01G000200LC,140,stress,0.5
17.7845964,all,TraesCS1A01G000300,140,stress,0.5
0.3985324,all,TraesCS1A01G000300LC,140,stress,0.5


In [12]:
unique(means_df$factor)

[1] "all"                                      
 [2] "PAMP chitin"                              
 [3] "PAMP flg22"                               
 [4] "Phosphorous starvation 10 d shoots"       
 [5] "powdery mildew"                           
 [6] "stripe rust 2"                            
 [7] "drought&heat combined stress"             
 [8] "heat stress"                              
 [9] "drought stress"                           
[10] "Fusarium pseudograminearum study2"        
[11] "PEG 6000"                                 
[12] "Zymoseptoria tritici"                     
[13] "Fusarium pseudograminearum"               
[14] "cold 2 weeks (4C)"                        
[15] "stripe rust"                              
[16] "all_means"                                
[17] "all_mean_filter"                          
[18] "PAMP control"                             
[19] "mildew stripe rust 2 control"             
[20] "Phosphorous starvation shoots control"    
[21] "Fusarium pseudograminearum control study2"
[22] "Zymoseptoria tritici control"             
[23] "Fusarium pseudograminearum control"       
[24] "stripe rust control"                      
[25] "heat drought control"                     
[26] "PEG 6000 control"                         
[27] "cold 2 weeks control"                     
[28] "No stress control"                        
[29] "spike"                                    
[30] "spikelets"                                
[31] "awns"                                     
[32] "glumes"                                   
[33] "anther"                                   
[34] "stigma & ovary"                           
[35] "rachis"                                   
[36] "microspores"                              
[37] "roots"                                    
[38] "root apical meristem"                     
[39] "seedling aerial tissues"                  
[40] "shoot apical meristem"                    
[41] "leaf blades excl flag"                    
[42] "leaf sheaths excl flag"                   
[43] "shoot axis"                               
[44] "flag leaf blade"                          
[45] "leaf ligule"                              
[46] "flag leaf sheath"                         
[47] "internode"                                
[48] "peduncle"                                 
[49] "leaf excl flag leaf"                      
[50] "flag leaf"                                
[51] "vegetative aerial tissues"                
[52] "grain milk and soft dough"                
[53] "grain hard dough and ripening"            
[54] "endosperm"                                
[55] "embryo"                                   
[56] "aleurone"                                 
[57] "seed coat"                                
[58] "endosperm + seed coat"                    
[59] "transfer cells"                           
[60] "aleurone layer and starchy endosperm"

In [11]:
saveRDS(means_df, file="./TablesForExploration/MeanTpms.rds")